# 导入所需模块

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


---

在图像处理的过程中，经常需要从图像中将前景对象作为目标图像分割或者提取出来。例如，在视频监控中，观测到的是固定背景下的视频内容，而我们对背景本身并无兴趣，感兴趣的是背景中出现的车辆、行人或者其他对象。我们希望将这些对象从视频中提取出来，而忽略那些没有对象进入背景的视频内容。
# 用分水岭算法实现图像分割与提取
图像分割是图像处理过程中一种非常重要的操作。分水岭算法将图像形象地比喻为地理学上的地形表面，实现图像分割，该算法非常有效。

## 算法原理
冈萨雷斯在《数字图像处理》一书中，对分水岭算法进行了细致的分析与介绍。OpenCV的官网建议学习者阅读国立巴黎高等矿业学校图像处理实验室（TheImage Processing Laboratory of MINES ParisTech）的CMM（Centre forMathematical Morphology）网站上关于分水岭算法的介绍和动画演示。

任何一幅灰度图像，都可以被看作是地理学上的地形表面，灰度值高的区域可以被看成是山峰，灰度值低的区域可以被看成是山谷

如果我们向每一个山谷中“灌注”不同颜色的水（这里采用了OpenCV官网的表述，冈萨雷斯将灌注表述为在山谷中打洞，然后让水穿过洞以均匀的速率上升）。那么，随着水位不断地升高，不同山谷的水就会汇集到一起。在这个过程中，为了防止不同山谷的水交汇，我们需要在水流可能汇合的地方构建堤坝。该过程将图像分成两个不同的集合：集水盆地和分水岭线。我们构建的堤坝就是分水岭线，也即对原始图像的分割。这就是分水岭算法。

由于噪声等因素的影响，采用上述基础分水岭算法经常会得到过度分割的结果。过度分割会将图像划分为一个个稠密的独立小块，让分割失去了意义。

为了改善图像分割效果，人们提出了基于掩模的改进的分水岭算法。改进的分水岭算法允许用户将他认为是同一个分割区域的部分标注出来（被标注的部分就称为掩模）。这样，分水岭算法在处理时，就会将标注的部分处理为同一个分割区域。大家可以尝试使用微软PowerPoint中的“删除背景”功能，加深对此改进算法的理解。

## 相关函数介绍
在OpenCV中，可以使用函数`cv2.watershed()`实现分水岭算法。在具体的实现过程中，还需要借助于形态学函数、距离变换函数`cv2.distanceTransform()`、`cv2.connectedComponents()`来完成图像分割。

### 形态学函数回顾
#### 开运算
开运算是先腐蚀、后膨胀的操作，开运算能够去除图像内的噪声。

#### 获取图像边界
通过形态学操作和减法运算能够获取图像的边界。



In [ ]:
o=cv2.imread("./images/rice.png",cv2.IMREAD_UNCHANGED)
k=np.ones((5,5),np.uint8)
e=cv2.erode(o,k)
b=cv2.subtract(o,e)
plt.subplot(131)
plt.imshow(o)
plt.axis('off')
plt.subplot(132)
plt.imshow(e)
plt.axis('off')
plt.subplot(133)
plt.imshow(b)
plt.axis('off') 

### 距离变换函数distanceTransform
当图像内的各个子图没有连接时，可以直接使用形态学的腐蚀操作确定前景对象，但是如果图像内的子图连接在一起时，就很难确定前景对象了。此时，借助于距离变换函数`cv2.distanceTransform()`可以方便地将前景对象提取出来。



In [ ]:
img = cv2.imread('./images/water_coins.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
ishow=img.copy()
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, fore = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
plt.subplot(131)
plt.imshow(ishow)
plt.axis('off')
plt.subplot(132)
plt.imshow(dist_transform)
plt.axis('off')
plt.subplot(133)
plt.imshow(fore)
plt.axis('off')


### 确定未知区域


In [ ]:
img = cv2.imread('./images/water_coins.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
ishow=img.copy()
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
bg = cv2.dilate(opening,kernel,iterations=3)
dist = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, fore = cv2.threshold(dist,0.7*dist.max(),255,0)
fore = np.uint8(fore)
un = cv2.subtract(bg,fore)
plt.subplot(221)
plt.imshow(ishow)
plt.axis('off')
plt.subplot(222)
plt.imshow(bg)
plt.axis('off')
plt.subplot(223)
plt.imshow(fore)
plt.axis('off')
plt.subplot(224)
plt.imshow(un)
plt.axis('off')


### 函数connectedComponents
明确了确定前景后，就可以对确定前景图像进行标注了。在OpenCV中，可以使用函数`cv2.connectedComponents()`进行标注。该函数会将背景标注为0，将其他的对象使用从1开始的正整数标注。



In [ ]:
img = cv2.imread('./images/water_coins.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
ishow=img.copy()
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
sure_bg = cv2.dilate(opening,kernel,iterations=3)
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, fore = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
fore = np.uint8(fore)
ret, markers = cv2.connectedComponents(fore)
plt.subplot(131)
plt.imshow(ishow)
plt.axis('off')
plt.subplot(132)
plt.imshow(fore)
plt.axis('off')
plt.subplot(133)
plt.imshow(markers)
plt.axis('off')
print(ret)


In [ ]:
img = cv2.imread('./images/water_coins.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
ishow=img.copy()
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
sure_bg = cv2.dilate(opening,kernel,iterations=3)
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, fore = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
fore = np.uint8(fore)
ret, markers1 = cv2.connectedComponents(fore)
foreAdv=fore.copy()
unknown = cv2.subtract(sure_bg,foreAdv)
ret, markers2 = cv2.connectedComponents(foreAdv)
markers2 = markers2+1
markers2[unknown==255] = 0
plt.subplot(121)
plt.imshow(markers1)
plt.axis('off')
plt.subplot(122)
plt.imshow(markers2)
plt.axis('off')


### 函数`cv2.watershed()`
完成上述处理后，就可以使用分水岭算法对预处理结果图像进行分割了。在OpenCV中，实现分水岭算法的函数是`cv2.watershed()`


## 分水岭算法图像分割实例
使用分水岭算法进行图像分割时，基本的步骤为：
1. 通过形态学开运算对原始图像O去噪。
2. 通过腐蚀操作获取“确定背景B”。需要注意，这里得到“原始图像-确定背景”即可。
3. 利用距离变换函数`cv2.distanceTransform()`对原始图像进行运算，并对其进行阈值处理，得到“确定前景F”。
4. 计算未知区域UN（UN=O -B - F）。
5. 利用函数`cv2.connectedComponents()`对原始图像O进行标注。
6. 对函数`cv2.connectedComponents()`的标注结果进行修正。
7. 使用分水岭函数完成对图像的分割

In [ ]:
img = cv2.imread('./images/water_coins.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
ishow=img.copy()
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
sure_bg = cv2.dilate(opening,kernel,iterations=3)
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
ret, markers = cv2.connectedComponents(sure_fg)
markers = markers+1
markers[unknown==255] = 0
markers = cv2.watershed(img,markers)
img[markers == -1] = [0,255,0]
plt.subplot(121)
plt.imshow(ishow)
plt.axis('off')
plt.subplot(122)
plt.imshow(img)
plt.axis('off')


---
# 交互式前景提取
经典的前景提取技术主要使用纹理（颜色）信息，如魔术棒工具，或根据边缘（对比度）信息，如智能剪刀等完成。2004年，微软研究院（剑桥）的Rother等人在论文GrabCut: Interactive Foreground Extraction Using Iterated Graph Cuts中提出了交互式前景提取技术。他们提出的算法，仅需要做很少的交互操作，就能够准确地提取出前景图像。

GrabCut算法的具体实施过程。
1. 将前景所在的大致位置使用矩形框标注出来。值得注意的是，此时矩形框框出的仅仅是前景的大致位置，其中既包含前景又包含背景，所以该区域实际上是未确定区域。但是，该区域以外的区域被认为是“确定背景”。
2. 根据矩形框外部的“确定背景”数据来区分矩形框区域内的前景和背景。
3. 用高斯混合模型（Gaussians Mixture Model, GMM）对前景和背景建模。GMM会根据用户的输入学习并创建新的像素分布。对未分类的像素（可能是背景也可能是前景），根据其与已知分类像素（前景和背景）的关系进行分类。
4. 根据像素分布情况生成一幅图，图中的节点就是各个像素点。除了像素点之外，还有两个节点：前景节点和背景节点。所有的前景像素都和前景节点相连，所有的背景像素都和背景节点相连。每个像素连接到前景节点或背景节点的边的权重由像素是前景或背景的概率来决定。
5. 图中的每个像素除了与前景节点或背景节点相连外，彼此之间还存在着连接。两个像素连接的边的权重值由它们的相似性决定，两个像素的颜色越接近，边的权重值越大。
6. 完成节点连接后，需要解决的问题变成了一幅连通的图。在该图上根据各自边的权重关系进行切割，将不同的点划分为前景节点和背景节点。
7. 不断重复上述过程，直至分类收敛为止。



In [ ]:
o = cv2.imread('./images/lenacolor.png')
orgb=cv2.cvtColor(o,cv2.COLOR_BGR2RGB)
mask = np.zeros(o.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
rect = (50,50,400,400)
cv2.grabCut(o,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
ogc = o*mask2[:,:,np.newaxis]
ogc=cv2.cvtColor(ogc,cv2.COLOR_BGR2RGB)
plt.subplot(121)
plt.imshow(orgb)
plt.axis('off')
plt.subplot(122)
plt.imshow(ogc)
plt.axis('off')

In [ ]:
o= cv2.imread('./images/lenacolor.png')
orgb=cv2.cvtColor(o,cv2.COLOR_BGR2RGB)
mask = np.zeros(o.shape[:2],np.uint8)
bgd = np.zeros((1,65),np.float64)
fgd = np.zeros((1,65),np.float64)
rect = (50,50,400,500)
cv2.grabCut(o,mask,rect,bgd,fgd,5,cv2.GC_INIT_WITH_RECT)
mask2 = cv2.imread('./images/mask.png',0)
mask2Show = cv2.imread('./images/mask.png',-1)
m2rgb=cv2.cvtColor(mask2Show,cv2.COLOR_BGR2RGB)
mask[mask2 == 0] = 0
mask[mask2 == 255] = 1
mask, bgd, fgd = cv2.grabCut(o,mask,None,bgd,fgd,5,cv2.GC_INIT_WITH_MASK)
mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
ogc = o*mask[:,:,np.newaxis]
ogc=cv2.cvtColor(ogc,cv2.COLOR_BGR2RGB)
plt.subplot(121)
plt.imshow(m2rgb)
plt.axis('off')
plt.subplot(122)
plt.imshow(ogc)
plt.axis('off')


In [ ]:
o= cv2.imread('./images/lenacolor.png')
orgb=cv2.cvtColor(o,cv2.COLOR_BGR2RGB)
bgd = np.zeros((1,65),np.float64)
fgd = np.zeros((1,65),np.float64)
mask2 = np.zeros(o.shape[:2],np.uint8)
mask2[30:512,50:400]=3
mask2[50:300,150:200]=1
cv2.grabCut(o,mask2,None,bgd,fgd,5,cv2.GC_INIT_WITH_MASK)
mask2 = np.where((mask2==2)|(mask2==0),0,1).astype('uint8')
ogc = o*mask2[:,:,np.newaxis]
ogc=cv2.cvtColor(ogc,cv2.COLOR_BGR2RGB)
plt.subplot(121)
plt.imshow(orgb)
plt.axis('off')
plt.subplot(122)
plt.imshow(ogc)
plt.axis('off')